# Brovey Pansharpening

La transformation Brovey a été élaborée dans le but d'augmenter le contraste visuel au niveau des extrémités supérieure et inférieure de l'histogramme des données. Elle fait appel à une méthode qui multiplie chaque pixel multispectral ré-échantillonné par le rapport de l'intensité du pixel panchromatique correspondant sur la somme de toutes les intensités multispectrales. Cette transformation suppose que **la gamme spectrale couverte par l'image panchromatique est identique à celle recouverte par les canaux multispectraux.**

Dans la transformation Brovey, l'équation générale utilise le rouge, le vert et le bleu (RVB), ainsi que les canaux panchromatiques pour générer de nouveaux canaux RVB, par exemple :

$$RedOut = (RedIn  * Panchromatic) / (BlueIn + GreenIn + RedIn)$$
$$GreenOut = (GreenIn  * Panchromatic) / (BlueIn + GreenIn + RedIn)$$
$$BlueOut = (BlueIn  * Panchromatic) / (BlueIn + GreenIn + RedIn)$$


In [ ]:
# Import
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
# Path jusqu'à nos images
PATH = "../images/our-images/"

In [ ]:
"""
load_images : Chargement de nos images black&white, basse qualité couleur et originale
name        : Nom de l'image que l'on veut charger
ext         : Extension des images que l'on veut charger
return      : Les trois images chargées
"""
def load_images(name, ext):
    path = PATH + name
    bw = cv2.imread(path + "-bw." + ext, cv2.IMREAD_GRAYSCALE)
    ds = cv2.imread(path + "-ds." + ext)
    ds = cv2.cvtColor(ds, cv2.COLOR_BGR2RGB)
    original = cv2.imread(path + "-cropped." + ext)
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    return (bw, ds, original)

bw, ds, original = load_images("star-fort", "jpg")

In [ ]:
"""
display_img : Affiche une image
img         : L'image à afficher
return      : Rien
"""
def display_img(img):
    plt.figure(figsize = (12, 12))
    plt.imshow(img, cmap="gray")

In [ ]:
display_img(ds)
display_img(bw)

In [ ]:
"""
brovey : Applique l'algorithme de pansharpehing de Brovey
bw     : L'image en Black&White
ds     : L'image en DownSampled, basse qualité en couleur
return : L'image pansharpened
"""
def brovey(bw, ds):
    bw_height, bw_width = bw.shape
    ds_height, ds_width, _ = ds.shape
    
    ds = cv2.resize(ds, dsize=(bw_height, bw_width))
    ds = ds.astype(float)
    
    pan = np.zeros((bw_height,bw_width,3), np.uint8)

    channels_sum = ds.sum(axis=2) / 3
    channels_sum[channels_sum == 0] = 1

    pan[:,:,0] = (ds[:,:,0] * bw) / channels_sum
    pan[:,:,1] = (ds[:,:,1] * bw) / channels_sum
    pan[:,:,2] = (ds[:,:,2] * bw) / channels_sum

    return pan    

In [ ]:
pan = brovey(bw, ds)

In [ ]:
display_img(pan)
display_img(original)

In [ ]:
"""
diff         : Compare l'image obtenue grâce au pansharpening avec l'image d'origine sur les trois canaux R, G, B
original     : L'image originale
pansharpened : L'image obtenue grâce au pansharpening
return       : Rien
"""
def diff(original, pansharpened):
    original = original.astype(int)
    pansharpened = pansharpened.astype(int)
    difference = 255 - abs(original - pansharpened)
    
    # Red
    display_img(difference[:,:,0])
    # Green
    display_img(difference[:,:,1])
    # Blue
    display_img(difference[:,:,2])

In [ ]:
difference = diff(original, pan)